# Food in Art

In [ ]:
from analysis import *
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler



## Loading

In [ ]:
config = load_config('config.yaml')


In [ ]:
all_data = load_all_datasets(config)

#### Load correspondance table

In [ ]:
correspondance_data = all_data['ids']
correspondance_data.drop_duplicates(inplace=True)
correspondance_data

#### Load paintings data

In [ ]:
paintings_data = all_data['paintings']
paintings_data['image_url'] = paintings_data['image_url'].apply(lambda x: get_512px_thumbnail(x) if pd.notna(x) else x)
paintings_data

#### Load locations data

In [ ]:
locations_data = all_data['locations']
locations_data

#### Load authors data

In [ ]:
authors_data = all_data['authors']
authors_data

#### Load ML food data

In [ ]:
food_words = all_data['food_words']
food_words['food_word_detected'] = food_words.select_dtypes(include='int').sum(axis=1) > 0
food_words

In [ ]:
food_found = all_data['food_found']
food_found['food_image_detected'] = food_found['predictions'].apply(lambda x: len(x) > 3)
food_found

### Merging

In [ ]:
merged_df = correspondance_data.merge(paintings_data, on='painting_id', how='inner')
merged_df = merged_df.merge(authors_data, on='author_id', how='left')
merged_df = merged_df.merge(locations_data, on='location_id', how='left')
merged_df = merged_df.merge(food_words, on='painting_id', how='left')
merged_df = merged_df.merge(food_found, on='painting_id', how='left')
merged_df

### Cleaning

### Duplicates

In [ ]:
merged_df = merged_df.drop_duplicates(subset='painting_id', keep='first')

### Merging

In [ ]:
merged_df['food_detected'] = (merged_df['food_word_detected'] | merged_df['food_image_detected']).astype(int)
merged_df

### Pruning

In [ ]:
merged_df = merged_df[merged_df['image_path'].notna()]

In [ ]:
print(merged_df['food_detected'].value_counts())
print(merged_df['food_word_detected'].value_counts())
print(merged_df['food_image_detected'].value_counts())

### Enhancing

In [ ]:

display(merged_df[['creation_date','date_of_birth']])
display(merged_df[['creation_date','date_of_birth']].describe())


merged_df['creation_date'] = merged_df['creation_date'].apply(extract_year)
merged_df['date_of_birth'] = merged_df['date_of_birth'].apply(extract_year)


display(merged_df[['creation_date','date_of_birth']])
display(merged_df[['creation_date','date_of_birth']].describe())

In [ ]:
# Fill missing creation year when possible
# Calculate the age of the painter at the time of painting
merged_df['painter_age_at_painting'] = merged_df['creation_date'] - merged_df['date_of_birth']

display(merged_df[['painter', 'creation_date', 'date_of_birth', 'painter_age_at_painting']])

# Calculate the average painter_age_at_painting for each painter
avg_painter_age = merged_df['painter_age_at_painting'].mean().astype(int)

merged_df['painter_age_at_painting'].fillna(avg_painter_age, inplace=True)

# Fill missing creation_date with date_of_birth + avg_painter_age
merged_df['creation_date'].fillna(merged_df['date_of_birth'] + avg_painter_age, inplace=True)

display(merged_df[['painter', 'creation_date', 'date_of_birth', 'painter_age_at_painting']])
merged_df

In [ ]:
merged_df['painter'].fillna('Unknown Artist', inplace=True)
merged_df['author_country'].fillna('Unknown Country', inplace=True)
merged_df['location_country'].fillna('Unknown Country', inplace=True)
merged_df['location_name'].fillna('Unknown Location', inplace=True)

merged_df['author_gender'] = merged_df['author_gender'].astype('category')
merged_df['author_gender'] = merged_df['author_gender'].cat.set_categories(['male', 'female'])
merged_df['author_gender'].fillna('male', inplace=True)

#### Add decades

In [ ]:

# Add a column with decades
merged_df['decade'] = (merged_df['creation_date'] // 10) * 10

# Display the updated DataFrame
display(merged_df[['painter', 'creation_date', 'decade']])
print(merged_df['decade'].unique())

In [ ]:
merged_df['location_country'] = merged_df['location_country'].replace('German Reich', 'Germany')
merged_df['author_country'] = merged_df['author_country'].replace('German Reich', 'Germany')

#### Add time period

In [ ]:

merged_df['time_period'] = merged_df['decade'].apply(classify_period)


#### Add gdp and pop

In [ ]:
eco_df = pd.read_csv('data/gdp_pop_decades.csv')
eco_df

In [ ]:
merged_df = merged_df.merge(
    eco_df,
    on='decade',
    how='left' 
)

merged_df

In [ ]:

scaler = MinMaxScaler()

# Normalize 'gdppc' and 'pop' columns
merged_df[['gdppc_normalized', 'pop_normalized']] = scaler.fit_transform(merged_df[['gdppc', 'pop']])

display(merged_df[['gdppc', 'gdppc_normalized', 'pop', 'pop_normalized']])

## FINAL DF

In [ ]:
paintings_with_food = merged_df[merged_df['image_url'].isna() == False]
paintings_with_food = paintings_with_food[['title', 'painter', 'creation_date', 'author_gender', 'author_country', 'location_name', 'location_country', 'time_period', 'image_path', 'image_url', 'coordinates','food_detected','decade','gdppc','pop', 'gdppc_normalized', 'pop_normalized']]
paintings_with_food

## Export

In [ ]:
paintings_with_food.to_csv('data/paintings_with_food.csv', index=False)

### GDP analysis

In [ ]:
# Group by decade and calculate the proportion of food_detected
food_by_decade = merged_df.groupby('decade')['food_detected'].agg(artwork_count='count', food_related_sum='sum').reset_index()
food_by_decade['proportion_food_detected'] = food_by_decade['food_related_sum'] / food_by_decade['artwork_count']

# Merge normalized GDP and population data
food_by_decade = food_by_decade.merge(
    merged_df[['decade', 'gdppc_normalized', 'pop_normalized']].drop_duplicates(),
    on='decade',
    how='left'
)

# Filter the DataFrame to include only records from 1250 to 2000
food_by_decade = food_by_decade[(food_by_decade['decade'] >= 1250) & (food_by_decade['decade'] <= 2000)]


In [ ]:
data = food_by_decade

# Calculate Pearson correlation
correlation = data['proportion_food_detected'].corr(data['gdppc_normalized'])
correlation_pvalue = stats.pearsonr(data['proportion_food_detected'], data['gdppc_normalized'])

# Calculate summary statistics
summary_stats = {
    'Pearson Correlation': correlation,
    'P-value': correlation_pvalue[1],
    'Sample Size': len(data),
    'Mean Food Proportion': data['proportion_food_detected'].mean(),
    'Mean GDP per Capita': data['gdppc_normalized'].mean(),
}

print("\nCorrelation Analysis Results:")
for key, value in summary_stats.items():
    print(f"{key}: {value:.4f}")
